# Semantic Segmentation using Eff-UNet on IDD

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"

import numpy as np
import tensorflow as tf
from IPython.display import clear_output
import os,sys,ntpath,fnmatch,shutil,cv2
from tensorflow import keras
from tensorflow.keras import backend as K
from numpy import asarray,zeros,moveaxis
import numpy as np
import pandas as pd
from os import path
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.initializers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
import os,sys,ntpath,fnmatch,shutil,cv2
import joblib,time,os.path,itertools
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from sklearn.metrics import *
from IPython.display import clear_output
from tqdm import tqdm_notebook,tqdm
from keras.models import load_model 
import gc
np.random.seed(0)
import warnings
warnings.filterwarnings("ignore")
clear_output()
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from time import time
import segmentation_models as sm
import tensorflow.keras
tensorflow.keras.backend.set_image_data_format('channels_last')

## Data Loading

In [3]:
# root path to save the trained model
model_root = "/media/ram/338f6363-03b7-4ad7-a2be-40c31f59dee4/20230418_backup/ram/Students/B.Tech_2020/sasank/trained_models/"

In [4]:
# path of the raw dataset, freshly extracted from the zip file
root = "/media/ram/338f6363-03b7-4ad7-a2be-40c31f59dee4/20230418_backup/ram/Students/B.Tech_2020/sasank/"

### Organising the dataset into a structured way

In [9]:
image_set = 1 # since training only first part of IDD
string_path = os.path.join(root ,"IDD_Segmentation")
print(string_path)

    
# Rename all file with file's folder_name as prefix
# parentFolder=string_path
# for dirName, subdirs, fileList in os.walk(parentFolder):    
#     for filename in fileList:
#         path = os.path.join(dirName, filename)
#         os.rename(path,"{}/{}_{}".format(dirName,ntpath.basename(dirName),ntpath.basename(filename)))
        
# Label Mask generation for given image_set. 
# The code for conversion of json to labels is from the public-code repository maintained by IDD owners
if image_set==1:
    !python "/media/ram/338f6363-03b7-4ad7-a2be-40c31f59dee4/20230418_backup/ram/Students/B.Tech_2020/sasank/Code/Training/lab113/public-code/preperation/createLabels.py") --datadir "/media/ram/338f6363-03b7-4ad7-a2be-40c31f59dee4/20230418_backup/ram/Students/B.Tech_2020/sasank/IDD_Segmentation" --id-type level1Id --color True --num-workers 2
# elif image_set==2:
#     !python "E:\Anime\public-code\preperation\createLabels.py" --datadir "C:\Users\sasan\OneDrive\Desktop\FYP\idd-segmentation\IDD_Segmentation" --id-type level3Id --color True --num-workers 2

# clear output Screen
clear_output(wait=True)

# Re-Organize files into a single folder. So that training is easier
def Organize_Labels(id_level):
    pattern,partial_dir='*level1Ids.png',"_label_level1"
    
    # Group files to corresponding train or val set
    def Group_Files(dirPath, pattern):
        for parentDir, dirnames, filenames in os.walk(dirPath):
            for filename in fnmatch.filter(filenames, pattern):
                shutil.move(os.path.join(parentDir, filename), r"{}{}/{}".format(dirPath,partial_dir,filename))
        return True
    
    # Make new Directory to Organize train Files
    os.mkdir(string_path+"/gtFine/train"+partial_dir)
    return_val = Group_Files(string_path+'/gtFine/train', pattern)
    
    # Make new Directory to Organize val Files
    os.mkdir(string_path+"/gtFine/val"+partial_dir)
    return_val = Group_Files(string_path+'/gtFine/val', pattern)
    
    return True

# Invoke Organize_Labels for Level1
# indicator=Organize_Labels(id_level=1)

# # Function to remove unwanted Files and Folders after mask generation
# def RemoveUnwantedFiles(dirPath, pattern):
#     for parentDir, dirnames, filenames in os.walk(dirPath):
#         for filename in fnmatch.filter(filenames, pattern):
#             shutil.move(os.path.join(parentDir, filename), dirPath+"/"+filename)
#         if len(os.listdir(parentDir)) == 0:
#             os.rmdir(parentDir)
#     return  True

# # Invoke RemoveUnwantedFiles for Image_set1
# if image_set==1:
#     indicator = RemoveUnwantedFiles(string_path+'/leftImg8bit/train', "*.png")
#     indicator = RemoveUnwantedFiles(string_path+'/leftImg8bit/val', "*.png")
#     indicator = RemoveUnwantedFiles(string_path+'/leftImg8bit/test', "*.png")

# # Invoke RemoveUnwantedFiles for Image_set2
# elif image_set==2:
#     indicator = RemoveUnwantedFiles(string_path+'/leftImg8bit/train', "*.jpg")
#     indicator = RemoveUnwantedFiles(string_path+'/leftImg8bit/val', "*.jpg")
#     indicator = RemoveUnwantedFiles(string_path+'/leftImg8bit/test', "*.jpg")
# print("Label generation successful for Image_Set part{} Done".format(image_set))


/media/ram/338f6363-03b7-4ad7-a2be-40c31f59dee4/20230418_backup/ram/Students/B.Tech_2020/sasank/IDD_Segmentation
/bin/bash: -c: line 0: syntax error near unexpected token `)'
/bin/bash: -c: line 0: `python "/media/ram/338f6363-03b7-4ad7-a2be-40c31f59dee4/20230418_backup/ram/Students/B.Tech_2020/sasank/Code/Training/lab113/public-code/preperation/createLabels.py") --datadir "/media/ram/338f6363-03b7-4ad7-a2be-40c31f59dee4/20230418_backup/ram/Students/B.Tech_2020/sasank/IDD_Segmentation" --id-type level1Id --color True --num-workers 2'


In [5]:
train_img_files1=sorted(os.listdir(root+'IDD_Segmentation/leftImg8bit/train'))
train_label_files1=sorted(os.listdir(root+'IDD_Segmentation/gtFine/train_label_level3'))
val_img_files1=sorted(os.listdir(root+'IDD_Segmentation/leftImg8bit/val'))
val_label_files1=sorted(os.listdir(root+'IDD_Segmentation/gtFine/val_label_level3'))
print(len(train_img_files1), len(train_label_files1), len(val_img_files1), len(val_label_files1))

6993 6993 981 981


## Data Preprocessing


### Image data preprocessing
- Resizing image data to 224x480x3
- Normalizing pixel values
- Save the changes

In [6]:
# import joblib
# import cv2
# from tqdm import tqdm

# def Prepare_Image_and_Save(path, name, img_files):
#     height, width = 224, 480
#     image = []
#     for j in range(len(path)):
#         for i in tqdm(range(len(img_files[j]))):
#             img = cv2.imread(path[j] + img_files[j][i])
#             img = cv2.resize(img, (width, height))
#             img = np.float32(img) / 255
#             image.append(img)
#     print(len(image))
#     joblib.dump(image, name)
#     return True

# path1 = [root+'IDD_Segmentation/leftImg8bit/train/']
# path2 = [root+'IDD_Segmentation/leftImg8bit/val/']
# train_img_files1=sorted(os.listdir(root+'IDD_Segmentation/leftImg8bit/train'))
# train_label_files1=sorted(os.listdir(root+'IDD_Segmentation/gtFine/train_label_level3'))
# val_img_files1=sorted(os.listdir(root+'IDD_Segmentation/leftImg8bit/val'))
# val_label_files1=sorted(os.listdir(root+'IDD_Segmentation/gtFine/val_label_level3'))

# img_files1,img_files2 = [train_img_files1],[val_img_files1]

# Indicator1 = Prepare_Image_and_Save(path1,root+'IDD_Segmentation/prep_train_img_files_save',img_files1)
# Indicator2 = Prepare_Image_and_Save(path2,root+'IDD_Segmentation/prep_val_img_files_save',img_files2)
# if Indicator1 and Indicator2: print("Data Preparation of Images Successful Done!")

### Label data preprocessing
- Resizing mask data to 224x480x26
- Performing one hot ecoding on mask
- Save the changes

In [7]:
# from scipy.sparse import csc_matrix
# def Prepare_Label_and_Save(path, n_classes, name):
#     sparse_list = []
#     for k in range(len(path)):
#         files = sorted(os.listdir(path[k]))
#         height, width, n_classes = 224, 480, n_classes
#         for j in tqdm(range(len(files))):
#             label = np.zeros((n_classes, height, width), dtype=np.uint8)
#             img = cv2.imread(path[k] + "/" + files[j], cv2.IMREAD_GRAYSCALE)
#             img1 = cv2.resize(img, (width, height))
#             for i in range(n_classes):
#                 label[i, :, :] = (img1 == i).astype(np.uint8)
#             sp_list = []
#             for i in range(label.shape[0]):
#                 sp_list.append(csc_matrix(label[i]))
#             sparse_list.append(sp_list)
#     joblib.dump(sparse_list, name)
#     return True


# path1 = [root+'IDD_Segmentation/gtFine/train_label_level3']
# path2 = [root+'IDD_Segmentation/gtFine/val_label_level3']

# img_files1,img_files2 = [train_img_files1],[val_img_files1]
# Indicator=Prepare_Label_and_Save(path1,26,root+"IDD_Segmentation/prep_train_label_files_save1")
# Indicator=Prepare_Label_and_Save(path2,26,root+"IDD_Segmentation/prep_val_label_files_save1")
# if Indicator1 and Indicator2: print("Data Preparation of Labels Successful Done!")

### Shuffling the data

In [8]:
# from os import path
# from sklearn.utils import shuffle
# import gc

# def Shuffle_Data(root=root+"IDD_Segmentation/"):
    
#     # Shuffle Prepared Image Data
#     path1, path3=root+"data/prep_train_img_files_save", root+"prep_train_label_files_save1"
#     if path.exists(path1) and path.exists(path3):
#         prep_train_img_files_save, prep_train_label_files_save=shuffle(joblib.load(path1),joblib.load(path3),random_state=0)
#         Dump=joblib.dump(prep_train_img_files_save, path1), joblib.dump(prep_train_label_files_save, path3)
#         del prep_train_img_files_save, prep_train_label_files_save
#         Junk= gc.collect() 

#     # Shuffle Prepared Label Mask Data
#     path2, path4=root+"data/prep_val_img_files_save", root+"prep_val_label_files_save1"
#     if path.exists(path2) and path.exists(path4):
#         prep_val_img_files_save, prep_val_label_files_save=shuffle(joblib.load(path2),joblib.load(path4),random_state=0)
#         Dump=joblib.dump(prep_val_img_files_save, path2), joblib.dump(prep_val_label_files_save, path4)
#         del prep_val_img_files_save, prep_val_label_files_save
#         Junk= gc.collect()
    
#     return True

# Indicator=Shuffle_Data()
# if Indicator:print("Data Shuffle Successful Done!")

### Train Test Split on full data

In [9]:
# from IPython.display import clear_output
# def Train_Test_Split(root=root, split=0.8):
#     total_no_samples=6993
#     split_index=(int((total_no_samples*0.8)/32))*32
    
#     path1, path3= root+"IDD_Segmentation/prep_train_img_files_save", root+"IDD_Segmentation/prep_train_label_files_save1"
#     prep_train_img_files_save, prep_train_label_files_save=joblib.load(path1), joblib.load(path3)
    
#     joblib.dump(prep_train_img_files_save[:split_index], root+"IDD_Segmentation/prep_train_img_files_save_80")
#     joblib.dump(prep_train_label_files_save[:split_index], root+"IDD_Segmentation/prep_train_label_files_save_80")
#     joblib.dump(prep_train_img_files_save[split_index:], root+"IDD_Segmentation/prep_test_img_files_save_20")
#     joblib.dump(prep_train_label_files_save[split_index:], root+"IDD_Segmentation/prep_test_label_files_save_20")
#     clear_output()
#     return True

# Indicator=Train_Test_Split()
# if Indicator:print("Data Train_Test_Split Successful Done!")

### Creating a train generator from prepared data

In [10]:
height,width,n_classes=224,480,26
def image_generator_j(start,end,saved_data):
    '''Function to get samples from saved data for each batch in each epoch'''
    return np.array(saved_data[start:end])

def label_generator_j(start,end,saved_data):
    '''Function to get Mask from saved data for each batch in each epoch'''
    slice_saved=saved_data[start:end]
    ar=np.empty((len(slice_saved),n_classes,height,width), dtype=np.uint8)
    for j in range(len(slice_saved)):
        for i in range(n_classes):
            ar[j][i]=slice_saved[j][i].todense()
    ar = moveaxis(ar, 1, 3)
    return ar

def train_batch_generator_j(batch_size,epochs):
    '''Function to yield train Images and Labels for each batch from saved Joblib file'''
    global prep_train_img_files_save, prep_train_label_files_save
    tr_L = len(prep_train_img_files_save)
    num_tr=0
    while num_tr<epochs*2 :
        train_batch_start=0
        train_batch_end = batch_size
        while train_batch_start < tr_L:
            train_limit = min(train_batch_end, tr_L)
            batch_x = image_generator_j(train_batch_start,train_limit,prep_train_img_files_save) 
            batch_y = label_generator_j(train_batch_start,train_limit,prep_train_label_files_save)
            yield (batch_x,batch_y)
            train_batch_start += batch_size   
            train_batch_end += batch_size
        num_tr+=1

def val_batch_generator_j(batch_size,epochs):
    '''Function to yield validation Images and Labels for each batch from saved Joblib file'''
    global prep_val_img_files_save, prep_val_label_files_save
    val_L = len(prep_val_img_files_save)
    num_val=0
    while num_val<epochs*2:
        val_batch_start=0
        val_batch_end = batch_size
        while val_batch_start < val_L:
            val_limit = min(val_batch_end, val_L)
            batch_valx = image_generator_j(val_batch_start,val_limit,prep_val_img_files_save)
            batch_valy = label_generator_j(val_batch_start,val_limit,prep_val_label_files_save)
            yield (batch_valx,batch_valy)
            val_batch_start += batch_size   
            val_batch_end += batch_size
        num_val+=1

In [11]:
train_batch_generator, val_batch_generator = train_batch_generator_j, val_batch_generator_j
prep_train_img_files_save, prep_train_label_files_save = joblib.load(root + "IDD_Segmentation/prep_train_img_files_save_80"), joblib.load(root + "IDD_Segmentation/prep_train_label_files_save_80")
prep_val_img_files_save, prep_val_label_files_save = joblib.load(root + "IDD_Segmentation/prep_val_img_files_save"), joblib.load(root + "IDD_Segmentation/prep_val_label_files_save1")

## Evaluation Metric

In [12]:
def Calculate_MIOU(y_val, y_pred):
    class_iou ,n_classes=[],26
    y_predi = np.argmax(y_pred, axis=3)
    y_truei = np.argmax(y_val, axis=3)
    for c in range(n_classes):
        TP = np.sum((y_truei == c) & (y_predi == c))
        FP = np.sum((y_truei != c) & (y_predi == c))
        FN = np.sum((y_truei == c) & (y_predi != c)) 
        IoU = TP / float(TP + FP + FN)
        if(float(TP + FP + FN) == 0):
            IoU=TP/0.001
        class_iou.append(IoU)
    MIoU=sum(class_iou)/n_classes
    return MIoU

def miou( y_true, y_pred ) :
    '''Funtion to Wraps a miou function into a TensorFlow op that executes when needed'''
    score = tf.py_function( lambda y_true, y_pred : Calculate_MIOU( y_true, y_pred).astype('float32'),[y_true, y_pred],'float32')
    return score

In [13]:
def Select_Model(weights_save_path=False):
     
    """
    Function to select a Model based on entered choice and load its respective Model weights for Successful Prediction 
    Input:  weights_save_path <Saved weights path or Boolean>
    Returns: Model <Model Object>       """

    import ntpath,segmentation_models as sm
    if weights_save_path==False:
        print("Model Weights not provided. Exiting...")
    Model = sm.Unet('efficientnetb7',classes=26,input_shape=(224, 480,3),activation='softmax')
    Model.load_weights(weights_save_path) if isinstance(weights_save_path, str) else Model.load_weights("")
    
    # m=int(input("    1.Restnet_50+U-Net    2.Unet    3.DeeplabV3    4.Pspnet    5.Segnet >> Enter a number to Choose a Model:\n>>> "))
    
    # if m==1:
        
    #     if weights_save_path==False:weights_save_path="/content/drive/My Drive/IID_Files1/New_Model_logs_save/Unet_imgnet_resnet50_nlrr.hdf5"
    #     sent, sm_weight=ntpath.basename(weights_save_path)[12:],""
    #     for ch in sent:
    #         if((ch >= 'a' and ch <= 'z') or (ch >= 'A' and ch <= 'Z')) or (ch >= '0' and ch <= '9'):
    #             sm_weight+=ch
    #         else: break
        
    #     Model = sm.Unet(sm_weight,classes=26,input_shape=(224, 480,3),activation='softmax')
    #     Model.load_weights(weights_save_path) if isinstance(weights_save_path, str) else Model.load_weights("/content/drive/My Drive/IID_Files1/New_Model_logs_save/Unet_imgnet_resnet50_nlrr.hdf5")
    
    # elif m==2:
    #     Model = Unet_Segmentation((224, 480,3), 26)
    #     Model.load_weights(weights_save_path) if isinstance(weights_save_path, str) else Model.load_weights("/content/drive/My Drive/IID_Files/Models_save/Unet.best.hdf5")


    # elif m==3:
    #     Model = Deeplab_V3((224, 480,3), 26) 
    #     Model.load_weights(weights_save_path) if isinstance(weights_save_path, str) else Model.load_weights("/content/drive/My Drive/IID_Files/Models_save/Deeplab.best.hdf5") 
    
    # elif m==4:
    #     Model=PSPNET((224, 480,3), 26)
    #     Model.load_weights(weights_save_path) if isinstance(weights_save_path, str) else Model.load_weights("/content/drive/My Drive/IID_Files/Models_save/Pspnet.best.hdf5") 
    
    # elif m==5:
    #     Model=Segnet_Segmentation((224,480,3), 26, 1)
    #     Model.load_weights(weights_save_path) if isinstance(weights_save_path, str) else Model.load_weights("/content/drive/My Drive/IID_Files/Models_save/Segnet.best.hdf5") 

    # else: raise Exception("Wrong Input>>Enter an Integer to choose a Model")
    
    return Model

def plot_segmentation(images, pred_labels, true_labels=[], plot_limit=5):

    """ 
    Function to plot Sub-plot of Image, Label and Model Output after prediction is performed
    Input  : (images, pred_labels, true_labels) <3dArray>, plot_limit <Int>
    Return : None """ 
    
    if (len(true_labels)==0):
        for i in range(images.shape[0]):
            if i==plot_limit:return 
            plt.figure(figsize=(14, 10))
            plt.subplot(1, 2, 1)
            plt.imshow(cv2.cvtColor(images[i],cv2.COLOR_BGR2RGB))
            plt.ylabel('Image',fontsize=16)
            plt.subplot(1, 2, 2)
            plt.imshow(color_code(pred_labels[i]))
            plt.ylabel('Prediction',fontsize=16)
            plt.tight_layout()
            plt.show()
    else:
        for i in range(images.shape[0]):
            if i==plot_limit:
                print("  "+" ------------------ "*8)
                return 
            plt.figure(figsize=(28, 20))
            plt.subplot(1,3,1)
            plt.imshow(cv2.cvtColor(images[i],cv2.COLOR_BGR2RGB))
            plt.ylabel('Image',fontsize=28)
            plt.subplot(1,3,2)
            plt.imshow(color_code(true_labels[i]))
            plt.ylabel('Label',fontsize=28)
            plt.subplot(1,3,3)
            plt.imshow(color_code(pred_labels[i]))
            plt.ylabel('Prediction',fontsize=28)
            plt.tight_layout()
            plt.show()


def plot_confusion_matrix(cm, normalize=True, title='Confusion matrix', cmap=plt.cm.Reds):

    ''' Function to plot Confusion Matrix for given 2D_Matrix ''' 

    # ref: https://github.com/scikit-learn/scikit-learn/issues/12700

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    classes_dict={'road': 0, 'parking': 1, 'drivable fallback': 2, 'sidewalk': 3, 'rail track': 4, 'non-drivable fallback': 5, 'person': 6, 'animal': 7, 'rider': 8, 'motorcycle': 9, 'bicycle': 10, 'autorickshaw': 11, 'car': 12, 'truck': 13, 'bus': 14, 'caravan': 15, 'trailer': 16, 'train': 17, 'vehicle fallback': 18, 'curb': 19, 'wall': 20, 'fence': 21, 'guard rail': 22, 'billboard': 23, 'traffic sign': 24, 'traffic light': 25}
    classes=list(list(classes_dict.keys()))
    plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title,fontsize=16)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90,fontsize=12)
    plt.yticks(tick_marks, classes,fontsize=12)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',fontsize=18)
    plt.xlabel('Predicted label',fontsize=18)
    plt.show()


def Print_result(Mean_MIoU, cf_matrix, Accuracy, true_labels_org, pred_labels_org, cr=True):

    ''' Function to print all Final computed Results '''

    # printing results
    print("\nPrinting Results:>>\n")
    print('-----------------------')
    print('|      MIOU Score     |')
    print('-----------------------')
    print('\n   MIOU Score: {}\n'.format(np.round(np.mean(Mean_MIoU),4)))

    print('-----------------------')
    print('|   Accuracy Score   |')
    print('-----------------------')
    print('\n   Accuracy Score: {}\n'.format(np.round(np.mean(Accuracy),4)))

    print('-----------------------')
    print('|  Confusion Matrix   |')
    print('-----------------------')
    plot_confusion_matrix(cf_matrix,normalize=True)
    
    if cr==True:
        print('-------------------------')
        print('| Classifiction Report  |')
        print('-------------------------')
        print("\n",classification_report(true_labels_org.ravel(),pred_labels_org.ravel()))
        print({'road': 0, 'parking': 1, 'drivable fallback': 2, 'sidewalk': 3, 'rail track': 4, 'non-drivable fallback': 5, 'person': 6, 'animal': 7, 'rider': 8, 'motorcycle': 9, 'bicycle': 10, 'autorickshaw': 11, 'car': 12, 'truck': 13, 'bus': 14, 'caravan': 15, 'trailer': 16, 'train': 17, 'vehicle fallback': 18, 'curb': 19, 'wall': 20, 'fence': 21, 'guard rail': 22, 'billboard': 23, 'traffic sign': 24, 'traffic light': 25})
    
    return 

def Load_For_Prediction(String):
    
    """
    Function to Load Serialized preprocessed Data of Train, Val and Test based on choice for Prediction
    Input  : String <String>
    Return : prep_train_img_files_save <X>, prep_train_label_files_save <Y> """

    global prep_train_img_files_save, prep_train_label_files_save

    if String =="Train_data":
        prep_train_img_files_save=joblib.load(root + "IDD_Segmentation/prep_train_img_files_save_80")
        prep_train_label_files_save=joblib.load(root + "IDD_Segmentation/prep_train_label_files_save_80")    
    elif String =="Val_data":
        prep_train_img_files_save=joblib.load(root+"IDD_Segmentation/prep_val_img_files_save")
        prep_train_label_files_save=joblib.load(root+"IDD_Segmentation/prep_val_label_files_save1")
    elif String =="Test_data":
        prep_train_img_files_save=joblib.load(root + "IDD_Segmentation/prep_test_img_files_save_20")
        prep_train_label_files_save=joblib.load(root + "IDD_Segmentation/prep_test_label_files_save_20")
    elif String == "Subset":
        prep_train_img_files_save=joblib.load(root + "hi_img")
        prep_train_label_files_save=joblib.load(root + "hi")
    else:
        raise Exception("Enter one of the string\n'Train_data' or 'Val_data' or 'Test_data'")
    
    return prep_train_img_files_save, prep_train_label_files_save

def plot_training_result(out):

    ''' Function to plot Epoch vs Crossentropy and Epoch vs MIOU graph after Traning of a Model ''' 

    # Sub_plot with two figures
    figure, loc_ind = plt.subplots(1, 2,figsize=(15,5))

    # Obtain values to plot 
    miou, val_miou= out.history['miou'], out.history['val_miou']
    loss, val_loss= out.history['loss'], out.history['val_loss']

    # epoch list
    num_epochs = list(range(1,len(miou)+1))

    # ploting Epoch vs Crossentropy Loss    
    loc_ind[0].plot(num_epochs,loss,'r',label='train loss',linewidth=1.25)
    loc_ind[0].plot(num_epochs,val_loss,'g',label='validation loss',linewidth=1.25)
    loc_ind[0].set_ylabel('Categorical Crossentropy Loss',fontsize=14)
    loc_ind[0].set_xlabel('Epoch',fontsize=14)
    loc_ind[0].set_title('Epoch vs Crossentropy Loss',fontsize=16)
    loc_ind[0].grid()
    loc_ind[0].legend()

    # Epoch vs Mean Intersection over union
    loc_ind[1].plot(num_epochs,miou,linestyle='--',marker='o',color="deeppink",label='train miou',linewidth=1.25)
    loc_ind[1].plot(num_epochs,val_miou,linestyle='--',marker='o',color="dodgerblue",label='val miou',linewidth=1.25)
    loc_ind[1].set_ylabel('Mean Intersection over union',fontsize=14)
    loc_ind[1].set_xlabel('Epoch',fontsize=14)
    loc_ind[1].set_title('Epoch vs Mean Intersection over union',fontsize=16)
    loc_ind[1].grid()
    loc_ind[1].legend()
    plt.tight_layout(pad=3.0)
    plt.show() 

def color_code(le_pred):

    """ 
    Function to Color Code given Label Mask using RGB-Color to make it suitable to Display  
    Input  : le_pred <2D_Array>
    Return : col_pred <3d_Array>   """
    
    col_pred = moveaxis(np.repeat(le_pred[:, :, np.newaxis], 3, axis=2), -1, 0)
    # print(col_pred)
    color_code = [
    [128, 64, 128], [72, 98, 91], [255, 204, 54], [220, 20, 60], [147, 114, 178],
    [132, 91, 83], [70, 70, 70], [105, 143, 35], [255, 69, 0], [0, 191, 255],
    [128, 0, 128], [255, 165, 0], [255, 0, 0], [0, 255, 0], [0, 0, 255],
    [255, 255, 0], [0, 255, 255], [255, 0, 255], [192, 192, 192], [255, 192, 203],
    [75, 0, 130], [255, 99, 71], [100, 149, 237], [255, 140, 0], [0, 255, 127],
    [255, 20, 147], [0, 250, 154]
]
         
    m, n= col_pred.shape[1], col_pred.shape[2]
    for k in range(3):
        for i in range(m):
            for j in range(n):
                index=7 if int(col_pred[k][i][j])==255 else int(col_pred[k][i][j])
                col_pred[k][i][j]=color_code[index][k]
    col_pred=np.moveaxis(col_pred, 0, -1)
    return col_pred


def image_prepare(data,batch_files,Model):
    
    '''Read and preprocess images to generete batch of images for prediction'''
    
    height,width,n_classes,image=Model.input_shape[1],480,26,[]
    for img_i in range(len(batch_files)):
        img = cv2.imread(data+batch_files[img_i])
        img = cv2.resize(img,(width,height))
        img = np.float32(img)/255
        image.append(img)
    image=np.array(image)
    return image


def image_prepare_j(slice_saved,Model):
    '''Function to get samples batch wise from saved data for Prediction'''
    return np.array(slice_saved)


def label_prepare_j(slice_saved,Model):
    
    '''Function to get Mask batch wise from saved data for Prediction'''
    
    height,width,n_classes=Model.input_shape[1], 480, 26
    ar=np.empty((len(slice_saved),n_classes,height,width), dtype=np.uint8)
    for j in range(len(slice_saved)):
        for i in range(n_classes):
            ar[j][i]=slice_saved[j][i].todense()
    ar = moveaxis(ar, 1, 3)
    return ar


def label_prepare(data,batch_files,Model):
    
    '''Read and preprocess Label Mask to generete batch of Labels for prediction'''
    
    height,width,n_classes,labels=Model.input_shape[1],480,26,[]
    for i in range(len(batch_files)):
        label = np.zeros((height, width, n_classes)).astype(np.uint8)
        img = cv2.imread(data+batch_files[i])
        img = cv2.resize(img,(width,height))
        img1 = img[:,:,0]
        for i in range(n_classes):
            label[:,:,i] = (img1==i).astype(np.uint8)
        labels.append(label)
    labels=np.array(labels)
    return labels


def prob_to_label(cf_matrix, Accuracy, predictions, label=False):
    
    ''' Function to get Actual Labels from Predicted probabilities to compute confusion matrix and Accuracy '''

    pred_Out,true_Out,pred_labels,true_labels=[],[],[],[]
    predictions = moveaxis(predictions[np.newaxis,:, :,:] if (len(predictions.shape)==3) else predictions, 3, 1)
    if isinstance(label,np.ndarray):label = moveaxis(label[np.newaxis,:, :,:] if (len(label.shape)==3) else label, 3, 1)
    
    for p_index in range(predictions.shape[0]):

        p1 = np.where(predictions[p_index]<0.5, predictions[p_index], 1)# if a[ijk]>=0.5 then a[ijk]=1 (False)
        p1 = np.where(p1==1, p1, 0).astype(int)
        w = np.argmax(p1, axis=0).astype(int) # 248*480 matrix with 1-6
        
        if isinstance(label,np.ndarray):

            p2= np.where(label[p_index]==1, label[p_index], 0)
            v = np.argmax(label[p_index], axis=0).astype(int)   # 224*480  
            Accuracy.append(np.round(accuracy_score(v.ravel(), w.ravel()),4))

            cf_matrix=np.add(cf_matrix, confusion_matrix(v.ravel(), w.ravel(),labels=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]))
            true_labels.append(v),true_Out.append(p2)          
        
        pred_labels.append(w),pred_Out.append(p1)
    
    if isinstance(label,np.ndarray):true_Out=moveaxis(np.array(true_Out), 1, 3)
    pred_Out=moveaxis(np.array(pred_Out), 1, 3)
    
    return pred_Out, pred_labels, true_Out, true_labels, cf_matrix, Accuracy


def Intersect_over_union(y_val, y_pred, Mean_MIoU):
    
    """
    Function to compute Intersect_Over_Union for given set of Samples Inputs 
    Input  : y_val <4D_Array>, y_pred <4D_Array>, Mean_MIoU <List>
    Return : Mean_MIoU <List>      """ 

    for index in range(y_pred.shape[0]):

        class_iou ,n_classes=[],26
        y_predi = np.argmax(y_pred[index], axis=2) 
        y_truei = np.argmax(y_val[index], axis=2)
        
        for c in range(n_classes):
            TP = np.sum((y_truei == c) & (y_predi == c))
            FP = np.sum((y_truei != c) & (y_predi == c))
            FN = np.sum((y_truei == c) & (y_predi != c)) 
            IoU = TP / (TP + FP + FN) if (TP + FP + FN)>0 else 0
            class_iou.append(IoU)
        
        MIoU=sum(class_iou)/n_classes
        Mean_MIoU.append(MIoU)
    
    return Mean_MIoU

In [14]:
def Function_1(path_i,Mean_MIoU=None,cf_matrix=None,Accuracy=None,Model=None,load=True,Read_from_File=True,plot=True,path_l=0,plot_limit=2):

    '''
    ----------------------------------------------------------------------------
    The Function contains the entire Deep Learning pipeline 
    Function predicts the output for given raw input

    1. Reading Image data files from a specified directory.
    2. Preprocessing Images by Resizing and Normalization.
    3. Preparation of Data to make it suitable for prediction.
    4. Deep learning Model Prediction for prepared input.
    5. Finally Plotting of the predicted image segmentation.  
    ----------------------------------------------------------------------------
        Parameters
        ----------
        path_i <string>                 : Absolute Path of images Data files
        Read_from_File <Boolean>        : Indicator to specify input format
        plot_limit <Int>                : Plot limit of segmentaion output
        Model <Int>                     : Model Choice for prediction
        Other Parameters	            : **kwargs other properties
    
        returns 
        --------
        Output Arguments             : Output values
    ----------------------------------------------------------------------------

    '''

    # Image resolution and classes
    height, width, n_classes = 224, 480, 7
    
    # Load selected Deep Learning model
    if load: Model = Select_Model()
    
    # Function to predict Batch wise to avoid run out of memory
    def Predict_Segment(i,data,lab,pb_size,Mean_MIoU,cf_matrix,Accuracy):
        
        '''  Function to perform prediction for a given subset of Data at a time  '''
        
        # Get index offset for data 
        s, e = i, min(i+pb_size,len(data))        
        
        if Read_from_File == True: # Image Data preparation
            test_images = image_prepare(path_i,data[s:e],Model)
            y_true = label_prepare(path_l,lab[s:e],Model) if isinstance(path_l, str) else [0]   #decompress sparce
        else:
            test_images = image_prepare_j(data[s:e],Model)
            y_true = label_prepare_j(lab[s:e],Model) if isinstance(path_l, list) else [0]
        
        # Model Prediction 
        y_pred = Model.predict(test_images)  
        
        # Get Actual Labels from Predicted probabilities and compute some metrics if label is avaliable
        pred_Out,pred_labels,true_Out,true_labels,cf_matrix,Accuracy = prob_to_label(cf_matrix,Accuracy,y_pred,y_true)
        if isinstance(path_l, str) or isinstance(path_l, list):
            Mean_MIoU = Intersect_over_union(np.array(true_Out),np.array(pred_Out),Mean_MIoU)

        if plot: # ploting Few Segmentation Samples from prediction 
          if ((i==0) or ((pb_size==1) and i<plot_limit)):
            if (i==0):print("\nFew Segmentation Samples:>>>\n")
            plot_segmentation(test_images,pred_labels,true_labels,plot_limit) 
        
        return pred_Out, true_Out,pred_labels,true_labels,cf_matrix,Mean_MIoU,Accuracy
    
    if Read_from_File: # Reading all Data files from specified Path 
        data = sorted(os.listdir(path_i))
        lab = sorted(os.listdir(path_l)) if (isinstance(path_l, str) or isinstance(path_l, list)) else 0
    else:
        data, lab = path_i,path_l if (isinstance(path_l, str) or isinstance(path_l, list)) else 0
    
    # Computing subset size for samples for prediction 
    pb_size=1 if (len(data)<plot_limit) or Model.name=="SEGNET" else plot_limit
    Pred_org,y_true_org,pred_labels_org,true_labels_org = [],[],[],[]

    # Invoke Predict_Segment() for prediction for many subset of Samples
    for i in range(0,len(data),pb_size):
        dat = Predict_Segment(i,data,lab,pb_size,Mean_MIoU,cf_matrix,Accuracy)
        Pred_org.extend(dat[0]),y_true_org.extend(dat[1])
        pred_labels_org.extend(dat[2]),true_labels_org.extend(dat[3])
        cf_matrix = dat[4]
    
    return np.array(pred_labels_org,dtype=np.uint8),np.array(true_labels_org,dtype=np.uint8),dat[4],dat[5],dat[6]

def Function_2(path_img,path_lab,Mean_MIoU=None,cf_matrix=None,Accuracy=None,Model=None,plot=True,Read_from_File=True,load=True,cr=True,plot_limit=2):
    
    '''
    ---------------------------------------------------------------------------------------
    The Function implements the entire Deep Learning pipeline where it accepts 
    Images and Labels to predict output and calculates segmentation  metrics

    1. Accept Images and Labels data as input from the directory
    2. perform Preprocessing and Data Preparation
    3. Model Prediction for prepared Data as input
    4. Performance Calculation using multiple metics
    5. Finally printing and Plotting of the prediction results
    ---------------------------------------------------------------------------------------
        Parameters
        ----------
        path_img, path_lab <string>                    : Absolute Path of Images and labels.
        Read_from_File <Boolean>                       : Indicator to specify input format
        Mean_MIoU, cf_matrix, Accuracy <metrics>       : Variables to store prediction Scores
        plot_limit <Int>                               : Plot limit of segmentaion output
        Model <Int>                                    : Model Choice for prediction
        Other Parameters	                           : **kwargs other properties

        returns 
        --------
        Mean_MIoU, cf_matrix, Accuracy                 : resulting Performance Metrics
    ----------------------------------------------------------------------------------------

    '''

    # perform prediction by calling function1 
    pred_labels_org,true_labels_org,cf_matrix,Mean_MIoU,Accuracy = Function_1(path_img,Mean_MIoU,cf_matrix,Accuracy,Model,load,Read_from_File,plot,path_lab,plot_limit)
    
    # printing prediction result
    if plot: Print_result(Mean_MIoU,cf_matrix,Accuracy,true_labels_org,pred_labels_org,cr)
    
    return Mean_MIoU, cf_matrix, Accuracy

In [15]:
from tqdm import tqdm_notebook,tqdm
def predict_for(data_for_prediction, weights_save_path=False):
    
    """  General Function to perform prediction for the specified data split  """
    
    Mean_MIoU, Accuracy, cf_matrix=[], [], np.zeros((26,26))
    x, y = Load_For_Prediction(data_for_prediction)
    Model, Skip = Select_Model(weights_save_path), 2
    
    for d in tqdm_notebook(range(0,len(x),Skip)):

        if (d>=(len(x)-Skip)): 
            plot,_,_=True,clear_output(),print("Total number of samples in {0} : {1}".format(data_for_prediction,len(x)))
        else: plot=False
        
        Miou, cf_matrix, Accuracy=Function_2(x[d:d+Skip],y[d:d+Skip],Mean_MIoU,cf_matrix,Accuracy,Model,plot,False,False,False)
    collected = gc.collect()
    
    return Miou, Accuracy, cf_matrix

In [16]:
# from segmentation_models import Unet
# from segmentation_models import get_preprocessing
# from segmentation_models.losses import bce_jaccard_loss
# from segmentation_models.metrics import iou_score

# BACKBONE = 'efficientnetb7'
# ENCODER_WEIGHTS = 'imagenet'
# CLASSES = ['road', 'parking', 'drivable fallback', 'sidewalk', 'rail track', 'non-drivable fallback', 'person', 'animal', 'rider', 'motorcycle', 'bicycle', 'autorickshaw', 'car', 'truck', 'bus', 'caravan', 'trailer', 'train', 'vehicle fallback', 'curb', 'wall', 'fence', 'guard rail', 'billboard', 'traffic sign', 'traffic light', 'pole', 'polegroup', 'obs-str-bar-fallback', 'building', 'bridge', 'tunnel', 'vegetation', 'sky', 'fallback background', 'unlabeled']
# DEVICE = 'cuda'

# # create segmentation model with pretrained encoder
# preprocessing_fn = get_preprocessing(BACKBONE)
# model = Unet(BACKBONE, encoder_weights='imagenet', classes=len(CLASSES))
# model.summary()

In [18]:
! pip show tensorflow

Name: tensorflow
Version: 2.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/ram/.local/lib/python3.8/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


# Training with imagenet pretrained weights

In [ ]:
start_time = time()
batch_size, epochs = 2, 50
model = sm.Unet('efficientnetb7',encoder_weights='imagenet',classes=26,input_shape=(224, 480,3),activation='softmax')
# model.summary()
tensorboard, filepath = TensorBoard(log_dir=root+"logs/unet_img_efficientnetb7_nlrr{}".format(str(time())[:10])),model_root+"Unet_imgnet_efficientnetb7_nlrr.hdf5"
steps_per_epoch,validation_steps=int((len(train_img_files1))/batch_size),int((len(val_img_files1))/batch_size)
model.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy',miou])
es = EarlyStopping(monitor='val_miou', mode='max', verbose=1, patience=5)
checkpoint = ModelCheckpoint(filepath, monitor='val_miou', verbose=2, save_best_only=True, mode='max')
history_tf=model.fit_generator(train_batch_generator(batch_size,epochs), steps_per_epoch=steps_per_epoch, epochs=epochs, verbose=1, validation_data=val_batch_generator(batch_size,epochs),
                            validation_steps=validation_steps, callbacks=[checkpoint,es,tensorboard])
print("--- %s seconds ---" % (time() - start_time))

2024-04-25 15:06:05.987826: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3336 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:18:00.0, compute capability: 8.6


Epoch 1/50


2024-04-25 15:06:42.295356: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Defaulting to user installation because normal site-packages is not writeable


# Training results

In [ ]:
# plot_training_result(history_tf)

# Results on train data

In [ ]:
# Miou, Accuracy, cf_matrix = predict_for("Train_data","EffUNet38.hdf5")

# Results on val data

In [ ]:
Miou, Accuracy, cf_matrix = predict_for("Val_data","EffUNet38.hdf5")

# Results on test data

In [ ]:
Miou, Accuracy, cf_matrix = predict_for("Test_data","EffUNet38.hdf5")

# Results for Subset data of 50 images

In [ ]:
Miou, Accuracy, cf_matrix = predict_for("Subset","EffUNet38.hdf5")

## Ignore from here, only for reference

In [ ]:
def color_code(le_pred):

    """ 
    Function to Color Code given Label Mask using RGB-Color to make it suitable to Display  
    Input  : le_pred <2D_Array>
    Return : col_pred <3d_Array>   """
    
    col_pred = moveaxis(np.repeat(le_pred[:, :, np.newaxis], 3, axis=2), -1, 0)
    # print(col_pred)
    color_code = [
    [128, 64, 128], [72, 98, 91], [255, 204, 54], [220, 20, 60], [147, 114, 178],
    [132, 91, 83], [70, 70, 70], [105, 143, 35], [255, 69, 0], [0, 191, 255],
    [128, 0, 128], [255, 165, 0], [255, 0, 0], [0, 255, 0], [0, 0, 255],
    [255, 255, 0], [0, 255, 255], [255, 0, 255], [192, 192, 192], [255, 192, 203],
    [75, 0, 130], [255, 99, 71], [100, 149, 237], [255, 140, 0], [0, 255, 127],
    [255, 20, 147], [0, 250, 154]
]
         
    m, n= col_pred.shape[1], col_pred.shape[2]
    for k in range(3):
        for i in range(m):
            for j in range(n):
                index=7 if int(col_pred[k][i][j])==255 else int(col_pred[k][i][j])
                col_pred[k][i][j]=color_code[index][k]
    col_pred=np.moveaxis(col_pred, 0, -1)
    return col_pred

#### Testing color coding and visualisations

In [ ]:
# def Color_code_label(label_image, sns_code=False):
    
#     """
#     Function to color code a given Label Mask
#     input  : Mask<Array>,  Boolean(Indicator of color scheme)
#     return : color coded Mask <Array>
#     """
    
#     # Switch axis of Mask
#     cf_label_image=np.moveaxis(label_image, -1, 0)
#     # cmap = plt.get_cmap('tab10')
#     # colors = cmap(np.linspace(0, 1, 256))
#     # preparing color mappings
#     color_code = [
#     [128, 64, 128], [72, 98, 91], [255, 204, 54], [220, 20, 60], [147, 114, 178],
#     [132, 91, 83], [70, 70, 70], [105, 143, 35], [255, 69, 0], [0, 191, 255],
#     [128, 0, 128], [255, 165, 0], [255, 0, 0], [0, 255, 0], [0, 0, 255],
#     [255, 255, 0], [0, 255, 255], [255, 0, 255], [192, 192, 192], [255, 192, 203],
#     [75, 0, 130], [255, 99, 71], [100, 149, 237], [255, 140, 0], [0, 255, 127],
#     [255, 20, 147], [0, 250, 154]
# ]
#     # print(len(color_code))
    
#     # change value in each pixel to a specific color 
#     m, n= cf_label_image.shape[1], cf_label_image.shape[2]
#     for k in range(3):
#         for i in range(m):
#             for j in range(n):
#                 index=26 if int(cf_label_image[k][i][j])==255 else int(cf_label_image[k][i][j])
#                 cf_label_image[k][i][j]=color_code[index][k]
    
#     # Switch back axis of color-coded Mask
#     cf_label_image=np.moveaxis(cf_label_image, 0, -1)
#     return cf_label_image

# visualize a color-coded Mask 
# plot_mask=Get_image(root + "IDD_Segmentation/gtFine/train_label_level3/train_label_level3_30_733588_gtFine_labellevel3Ids.png")
# print(plot_mask.shape)
# cf_label_image=Color_code_label(plot_mask)
# plt.figure(figsize=(9.6, 6.8))
# plt.title("Color Coded Mask")
# plt.imshow(cf_label_image)
# plt.show()

#### Testing labels level of masks and intensities

In [ ]:

# # Get and plot label mask
# plt.figure(figsize=(9.6, 6.8))
# Label_Mask=Get_images('C:/Users/sasan/OneDrive/Desktop/FYP/idd-segmentation/testing/',True)
# plt.imshow(Label_Mask, cmap='gray')
# plt.title("Label Mask")
# plt.show()
# print("Label Mask Shape:",Label_Mask.shape)

# # compute Value Count of each pixel
# Label_Mask=[i for i in Label_Mask.ravel() if i != 255]
# (unique, counts) = np.unique(Label_Mask, return_counts=True)
# Value_count= dict(zip(unique, counts))
# print("\nUnique Class Labels in Mask:",set(Label_Mask))
# print("\nClass Value Count:",Value_count)

#### Test subset creation

##### Subset image creation

In [ ]:
# import joblib
# import cv2
# from tqdm import tqdm

# def Prepare_Image_and_Save(path, name, img_files):
#     height, width = 224, 480
#     image = []
#     for j in range(len(path)):
#         for i in tqdm(range(len(img_files[j]))):
#             print(path[j] + img_files[j][i])
#             img = cv2.imread(path[j] + img_files[j][i])
#             img = cv2.resize(img, (width, height))
#             img = np.float32(img) / 255
#             image.append(img)
#     print(len(image))
#     joblib.dump(image, name)
#     return True

# path1 = [root+'sample_test_img/']
# train_img_files1=sorted(os.listdir(root+'sample_test_img/'))

# img_files1 = [train_img_files1]

# Indicator1 = Prepare_Image_and_Save(path1,root+'hi_img',img_files1)
# if Indicator1: print("Data Preparation of Images Successful Done!")

##### Subset label creation

In [ ]:
from scipy.sparse import csc_matrix
def Prepare_Label_and_Save(path, n_classes, name):
    sparse_list = []
    for k in range(len(path)):
        files = sorted(os.listdir(path[k]))
        height, width, n_classes = 224, 480, n_classes
        for j in tqdm(range(len(files))):
            label = np.zeros((n_classes, height, width), dtype=np.uint8)
            img = cv2.imread(path[k] + "/" + files[j], cv2.IMREAD_GRAYSCALE)
            img1 = cv2.resize(img, (width, height))
            for i in range(n_classes):
                label[i, :, :] = (img1 == i).astype(np.uint8)
            sp_list = []
            for i in range(label.shape[0]):
                sp_list.append(csc_matrix(label[i]))
            sparse_list.append(sp_list)
    joblib.dump(sparse_list, name)
    return True


path1 = [root+'/sample_test_labels']

Indicator=Prepare_Label_and_Save(path1,26,root+"hi")
if Indicator: print("Data Preparation of Labels Successful Done!")